# Data Consolidation Notebook
This notebook is structured to facilitate the data consolidation and preparation of the diary.csv for further analysis. 

In [1]:
# Import necessary libraries
import pandas as pd  # for data manipulation and analysis
import requests  # to make HTTP requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects  # to handle potential request errors
import json  # to handle JSON data formats

## Initial Data Loading and Display
The following block loads the data from a specified source and displays the first few rows. This is crucial for understanding the initial structure of the data and ensuring that it has been loaded correctly.

In [2]:
# Load data from the CSV file and display the first few rows to inspect the data
df = pd.read_csv("data/diary.csv")
df.head()  # Display the first five rows of the dataframe

,Date,Name,Year,Letterboxd URI,Rating,Rewatch,Tags,Watched Date
0,5/3/21,Godzilla vs. Kong,2021,https://boxd.it/1PXghH,3.0,NaN,NaN,5/3/21
1,5/19/21,The Woman in the Window,2021,https://boxd.it/1S4UiR,2.5,NaN,NaN,5/19/21
2,5/24/21,About Time,2013,https://boxd.it/1SLTdH,4.5,NaN,NaN,5/24/21
3,5/26/21,Pretty Woman,1990,https://boxd.it/1T0ZVb,2.5,NaN,NaN,5/26/21
4,6/2/21,The Handmaiden,2016,https://boxd.it/1TZaH5,4.0,NaN,NaN,6/2/21
...,...,...,...,...,...,...,...,...
321,4/23/24,Galaxy Quest,1999,https://boxd.it/6kDYMZ,4.0,NaN,NaN,4/23/24
322,4/25/24,Misery,1990,https://boxd.it/6lmxKR,4.5,NaN,NaN,4/25/24
323,4/27/24,Hot Fuzz,2007,https://boxd.it/6m6EV5,4.0,Yes,NaN,4/27/24
324,4/27/24,Ricky Stanicky,2024,https://boxd.it/6m7Xpf,2.0,NaN,NaN,4/27/24


## Data Manipulation
Here we manipulate and clean the data. This section includes calling the TMDb API, filtering, handling missing values, and creating new tables, which are essential steps for preparing the data for analysis.

In [3]:
api_key = '53d52c8f3eb74eb1aa0e7b778fc96523' # API key for The Movie Database (TMDb)
url_base = "https://api.themoviedb.org/3/" # Base URL for TMDb API
session = Session() # Create a new session object to make requests

In [ ]:
tmdblist = []
for index,row in df.iterrows(): 
    title = row['Name']  # Adjust as necessary to match your DataFrame column name
    year = str(row['Year'])  # Convert year to string for comparison
    url = f"{url_base}search/movie?api_key={api_key}&query={title}"
    try:
        response = session.get(url)
        data = json.loads(response.text)
        results = data.get('results', [])
        if results:  # Check if results list is not empty
            # Loop through results to find a matching year
            for result in results:
                release_date = result.get('release_date', '')
                # Extract the year from the release_date
                release_year = release_date.split('-')[0] if release_date else ''
                if release_year == year: # Check if the release year matches the year in the DataFrame
                    movie_data = {
                        'movie_title': result.get('title', 'Title not found'),
                        'movie_id': result.get('id', 'ID not found'),
                        'language': result.get('original_language', 'Language not found'),
                        'poster': f"https://image.tmdb.org/t/p/original'{result.get('poster_path', '')}",
                        'popularity': result.get('popularity', 0),
                        'vote_average': result.get('vote_average', 0)
                    }
                    
                    tmdblist.append(movie_data)
                    break  # Stop searching after the first match

        if not tmdblist:
            print(f"No matching results found for title: {title} in year {year}")

    except (requests.ConnectionError, requests.Timeout, requests.TooManyRedirects) as e:
        print(e)

print('Finished extracting and matching movies from diary.csv')

There were some issues with the data that needed to be addressed:
- There were some missing values in the generated tmdb_diary.csv file. So we had to manually look up the missing values and fill them in. The following block fills in the missing values in the tmdb_diary.csv file.

Note that I reordered the cells in the notebook to make it easier to follow the data preparation process.

In [ ]:
# manually search and add missing movies to tmdb_diary when compare diary to tmdb_diary left join
missing_ids = ["302104","22970","181886","310131","661374","520763","537116","581734","522627","458156","339967","823464"]
for i in missing_ids: # Loop through the list of movie IDs we noticed were missing
    url = f"{url_base}/movie/{i}?api_key={api_key}&append_to_response=credits"

    try:
        response = session.get(url)
        if response.status_code == 200:
            data = response.json()
            movie_data = {
                'movie_title': data.get('title', 'Title not found'),
                'movie_id': data.get('id', 'ID not found'),
                'language': data.get('original_language', 'Language not found'),
                'poster': f"https://image.tmdb.org/t/p/original{data.get('poster_path', '')}",  # Fixed quote placement
                'popularity': data.get('popularity', 0),
                'vote_average': data.get('vote_average', 0)
            }
            tmdblist.append(movie_data)
            print(f"Manual entry {movie_data['movie_title']} successful.")
        else:
            print(f"Failed to fetch data for ID {i}: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"Error processing ID {i}: {str(e)}")

In [ ]:
tmdb_df = pd.DataFrame(tmdblist)
tmdb_df.to_csv('data/generated/tmdb_diary.csv', index=False)
tmdb_df

In [ ]:
# Merging the dataframes based on the movie title
combined_df = pd.merge(df, tmdb_df, left_on='Name', right_on='movie_title', how='left')
"""
Left Join because the important data here is our own diary.
This is also where I did some manual data cleaning because
the search function created wrong entries in `tmdb_diary.csv`,
so I had to search these values up manually. Some values simply
fall through the cracks.

In the next iteration of this project, I would like to implement
a better search function that is more robust and can handle
more edge cases. Perhaps joining on the movie title is not the
best way to do this, but I couldn't find a better way to do it based
on the data I had available.
"""
# # Drop duplicates with a specific condition that ignores valid re-watches
combined_df = combined_df.drop_duplicates(subset=['Name', 'movie_id', 'Watched Date'], keep='first')
# Save the cleaned-up DataFrame
combined_df.to_csv('data/generated/combined_diaries.csv', index=False)

Note: Remember to manually browse through the generated combined entries to ensure that the combined entries are correctly filled in.

In [ ]:
working_data = pd.read_csv('data/generated/combined_diaries.csv')
working_data

Let us now make another API call to get additional information about the movies in our dataset. We will use the movie_id from the previous API call to get more detailed information about each movie.

In [ ]:
# Initialize an empty list
ids = []

# Loop through 'movie_id' in working_data and add each to the set
for unique_id in working_data['movie_id']:
    ids.append(unique_id)

# print(ids_list)

In [ ]:
addtl_tmdb_list = []
for uid in ids:
    # https://api.themoviedb.org/3/movie/1700?api_key=2ca28daf3aadbe02b38f276d24d29e8e&append_to_response=credits
    url = f"{url_base}/movie/{uid}?api_key={api_key}&append_to_response=credits"
    try:
        response = session.get(url)
        if response.status_code == 200:
            data = response.json()
            genres = [genre['name'] for genre in data['genres']]
            addtl_movie_data = {
                'movie_title': data.get('title', 'Title not found'),
                'movie_id': data.get('id', 'ID not found'),
                'release_date': data.get('release_date', 'Release date not found'),
                'genres' : genres,
                'origin_country': data.get('origin_country', []),
                'revenue': data.get('revenue', 0),
                'runtime': data.get('runtime', 'Not specified'),
                'tagline': data.get('tagline', 'No tagline'),
                'cast': [{'name': member.get('name', 'Unknown'), 'gender': member.get('gender', 'Not specified')} for member in data.get('credits', {}).get('cast', [])],
                'director': [crew_member.get('name', 'Unknown') for crew_member in data.get('credits', {}).get('crew', []) if crew_member.get('job') == 'Director']
            }
            addtl_tmdb_list.append(addtl_movie_data)
        else:
            print(f"Failed to fetch data for ID {uid}: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"Error processing ID {uid}: {str(e)}")
print('Fetch Additional Details Success!')

Let's now export the additional information to a CSV file for further analysis and to ensure that we have a backup of the data.

In [ ]:
# Convert list of dictionaries to DataFrame
addtl_tmdb_df = pd.DataFrame(addtl_tmdb_list)

# Drop duplicates with a specific condition that ignores valid re-watches
addtl_tmdb_df = addtl_tmdb_df.drop_duplicates(subset=['movie_title','movie_id'], keep='first')
# Save the cleaned-up DataFrame
addtl_tmdb_df.to_csv('data/generated/addtl_tmdb_diaries.csv', index=False)


print('Finished extracting addtl_tmdb_diary.csv details!')

# Data Consolidation

Let's now combine the additional information with the existing data to create a consolidated dataset for further analysis.

Let us also standardize column names and data types to ensure consistency in the dataset, remove columns that are not needed, and handle incorrect values.

In [ ]:

# Merge working_data and addtl_tmdb_df using left join
working_combined_df = pd.merge(working_data, addtl_tmdb_df, how='left', left_on='Name', right_on='movie_title')

# Replace 'Name' column with 'movie_title_x' values
working_combined_df['Name'] = working_combined_df['movie_title_x']

# Drop 'Date', 'Tags', 'movie_title_x', 'movie_title_y' columns
columns_to_drop = ['Date', 'Tags', 'movie_title_x', 'movie_title_y','movie_id_y']
working_combined_df.drop(columns=columns_to_drop, inplace=True)

# Rename some columns
working_combined_df = working_combined_df.rename(columns={'movie_id_x': 'movie_id'})

# Standardize column names (convert to lowercase and replace spaces with underscores)
working_combined_df.columns = [col.lower().replace(' ', '_') for col in working_combined_df.columns]

# Convert 'watched_date' and 'release_date' to datetime format
# Might need to reformat some entries manually because on release_date, 48 == 1948, != 2048
working_combined_df['watched_date'] = pd.to_datetime(working_combined_df['watched_date'])
working_combined_df['release_date'] = pd.to_datetime(working_combined_df['release_date'])

# Order by 'watched_date'
working_combined_df.sort_values(by='watched_date', inplace=True)

We can use this consolidated dataset for further analysis and visualization through Python.

In [ ]:
working_combined_df.to_csv('data/generated/working_combined_diaries.csv', index=False)
print("f")
# This is the final csv for staging and EDA in Python. Left Joined on our diary.csv, and added additional information from tmdb

# Data Preparation for Tableau

We should also create a version of the dataset that will play nicely with Tableau. Especially since Tableau is not as flexible as Python in handling lists and dictionaries.

In [ ]:
df = pd.read_csv('data/generated/working_combined_diaries.csv')

The following block generates the df that we'll later split into separate tables for genre, country, and director list items

In [ ]:
# Function to split and clean a list string, handling NaN values, with dynamic column creation
def split_list_string(list_string, prefix):
    if pd.isna(list_string):
        return {}
    # Remove square brackets and split by comma
    elements = list_string.strip("[]").replace("'", "").split(", ")
    return {f"{prefix}_{i+1}": elements[i] for i in range(len(elements))}

# Apply the function to the 'genres' column
genres_df = df['genres'].apply(split_list_string, prefix='genre').apply(pd.Series)
df = df.drop(columns=['genres'])
df = pd.concat([df, genres_df], axis=1)

# Apply the function to the 'origin_country' column
origin_country_df = df['origin_country'].apply(split_list_string, prefix='country').apply(pd.Series)
df = df.drop(columns=['origin_country'])
df = pd.concat([df, origin_country_df], axis=1)

# Apply the function to the 'director' column
director_df = df['director'].apply(split_list_string, prefix='director').apply(pd.Series)
df = df.drop(columns=['director'])
df = pd.concat([df, director_df], axis=1)

# Remove columns with all NaN values
df = df.dropna(axis=1, how='all')

Let's add a column for the entry_id to the df so that we can join the tables later.

In [ ]:
# Add a unique id for each entry with standardized format of 5 digits
df['entry_id'] = df.index + 1
df['entry_id'] = df['entry_id'].apply(lambda x: f"{x:05d}")

# Add the id column to the beginning of the DataFrame
df = pd.concat([df['entry_id'], df.drop(columns=['entry_id'])], axis=1)
df

df.to_csv('data/generated/final_combined_diaries.csv', index=False)

The following block splits the df into separate tables for genre, country, and director list items. This is necessary for creating a relational database in Tableau without having to pivot through Tableau, making it easier to work with the data.

In [ ]:
# Create a separate table for genres where the values are pivoted; three columns only: entry id, genre number and genre sorted by entry id and genre number
df_entries_and_genres = df[['id','genre_1','genre_2','genre_3','genre_4','genre_5']].melt(id_vars='id', value_name='genre').dropna()
df_entries_and_genres['genre_number'] = df_entries_and_genres['variable'].str.extract(r'genre_(\d+)')
df_entries_and_genres = df_entries_and_genres.drop(columns='variable')
df_entries_and_genres = df_entries_and_genres.sort_values(by=['id','genre_number'])
df_entries_and_genres

# Save the updated DataFrame to a new CSV file
df_entries_and_genres.to_csv('data/generated/entries_and_genres.csv', index=False)

In [ ]:
# Create a separate table for countries where the values are pivoted; three columns only: entry id, country number and country sorted by entry id and country number
df_entries_and_countries = df[['id','country_1','country_2','country_3']].melt(id_vars='id', value_name='country').dropna()
df_entries_and_countries['country_number'] = df_entries_and_countries['variable'].str.extract(r'country_(\d+)')
df_entries_and_countries = df_entries_and_countries.drop(columns='variable')
df_entries_and_countries = df_entries_and_countries.sort_values(by=['id','country_number'])
df_entries_and_countries

# Save the updated DataFrame to a new CSV file
df_entries_and_countries.to_csv('data/generated/entries_and_countries.csv', index=False)

In [ ]:
# Create a separate table for directors where the values are pivoted; three columns only: entry id, director number and director sorted by entry id and director number
df_entries_and_directors = df[['id','director_1','director_2','director_3']].melt(id_vars='id', value_name='director').dropna()
df_entries_and_directors['director_number'] = df_entries_and_directors['variable'].str.extract(r'director_(\d+)')
df_entries_and_directors = df_entries_and_directors.drop(columns='variable')
df_entries_and_directors = df_entries_and_directors.sort_values(by=['id','director_number'])
df_entries_and_directors

# Save the updated DataFrame to a new CSV file
df_entries_and_directors.to_csv('data/generated/entries_and_directors.csv', index=False)

# These generated files will be used to create the database schema and tables for the project's final data visualization and analysis in Tableau.

Our data is now ready for further analysis and visualization in Tableau.